# EBAC - Regressão II - regressão múltipla

## Tarefa I

#### Previsão de renda II

Vamos continuar trabalhando com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

In [24]:
import pandas as pd
import math

import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import Ridge
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score


from scipy.stats import ks_2samp
import statsmodels.formula.api as smf
import statsmodels.api as sm
from statsmodels.formula.api import ols

import patsy

import scipy.stats

In [25]:
df = pd.read_csv('previsao_de_renda.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             15000 non-null  int64  
 1   data_ref               15000 non-null  object 
 2   id_cliente             15000 non-null  int64  
 3   sexo                   15000 non-null  object 
 4   posse_de_veiculo       15000 non-null  bool   
 5   posse_de_imovel        15000 non-null  bool   
 6   qtd_filhos             15000 non-null  int64  
 7   tipo_renda             15000 non-null  object 
 8   educacao               15000 non-null  object 
 9   estado_civil           15000 non-null  object 
 10  tipo_residencia        15000 non-null  object 
 11  idade                  15000 non-null  int64  
 12  tempo_emprego          12427 non-null  float64
 13  qt_pessoas_residencia  15000 non-null  float64
 14  renda                  15000 non-null  float64
dtypes:

1. Separe a base em treinamento e teste (25% para teste, 75% para treinamento).
2. Rode uma regularização *ridge* com alpha = [0, 0.001, 0.005, 0.01, 0.05, 0.1] e avalie o $R^2$ na base de testes. Qual o melhor modelo?
3. Faça o mesmo que no passo 2, com uma regressão *LASSO*. Qual método chega a um melhor resultado?
4. Rode um modelo *stepwise*. Avalie o $R^2$ na vase de testes. Qual o melhor resultado?
5. Compare os parâmetros e avalie eventuais diferenças. Qual modelo você acha o melhor de todos?
6. Partindo dos modelos que você ajustou, tente melhorar o $R^2$ na base de testes. Use a criatividade, veja se consegue inserir alguma transformação ou combinação de variáveis.
7. Ajuste uma árvore de regressão e veja se consegue um $R^2$ melhor com ela.

1. Separe a base em treinamento e teste (25% para teste, 75% para treinamento).

In [26]:
df = pd.read_csv('previsao_de_renda.csv')
df.drop(['Unnamed: 0', 'id_cliente', 'data_ref'], axis=1, inplace=True)
df.dropna(inplace=True)
df.head()

,sexo,posse_de_veiculo,posse_de_imovel,qtd_filhos,tipo_renda,educacao,estado_civil,tipo_residencia,idade,tempo_emprego,qt_pessoas_residencia,renda
0,F,False,True,0,Empresário,Secundário,Solteiro,Casa,26,6.602740,1.0,8060.34
1,M,True,True,0,Assalariado,Superior completo,Casado,Casa,28,7.183562,2.0,1852.15
2,F,True,True,0,Empresário,Superior completo,Casado,Casa,35,0.838356,2.0,2253.89
3,F,False,True,1,Servidor público,Superior completo,Casado,Casa,30,4.846575,3.0,6600.77
4,M,True,False,0,Assalariado,Secundário,Solteiro,Governamental,33,4.293151,1.0,6475.97


In [27]:
X = pd.get_dummies(df.drop('renda', axis=1), drop_first = True)
y = df['renda']
X.values.reshape(1,-1)
X_train, y_train, X_test, y_test = train_test_split(y, X, test_size = 0.25, train_size= 0.75, random_state = 42)

In [28]:
y_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3107 entries, 14812 to 9495
Data columns (total 24 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   posse_de_veiculo               3107 non-null   bool   
 1   posse_de_imovel                3107 non-null   bool   
 2   qtd_filhos                     3107 non-null   int64  
 3   idade                          3107 non-null   int64  
 4   tempo_emprego                  3107 non-null   float64
 5   qt_pessoas_residencia          3107 non-null   float64
 6   sexo_M                         3107 non-null   uint8  
 7   tipo_renda_Bolsista            3107 non-null   uint8  
 8   tipo_renda_Empresário          3107 non-null   uint8  
 9   tipo_renda_Pensionista         3107 non-null   uint8  
 10  tipo_renda_Servidor público    3107 non-null   uint8  
 11  educacao_Pós graduação         3107 non-null   uint8  
 12  educacao_Secundário            3107 non-null

2. Rode uma regularização *ridge* com alpha = [0, 0.001, 0.005, 0.01, 0.05, 0.1] e avalie o $R^2$ na base de testes. Qual o melhor modelo?

In [81]:
modelo = 'renda ~ sexo + posse_de_veiculo + posse_de_imovel + estado_civil + educacao + tipo_renda + tipo_residencia + tempo_emprego + idade + qtd_filhos'
md = smf.ols(modelo, data = df)
reg = md.fit_regularized(method = 'elastic_net' 
                         , L1_wt = 0.0
                         , alpha = 0.0
                         , refit = True)
print(reg.params) 


[-3089.46480903  6051.4090577     48.32488841   368.13095682
  -107.74151786  -440.71721953  -476.51082505  -217.1467775
  1089.54857268   607.39672789  1224.14661159   220.57285332
 -1358.46738384   820.78808191 -2686.81896167   138.02659474
  -297.8578571   -387.23789311  -458.99344929    79.52976065
    81.8553669    554.69422566    42.06813615   141.34354939]


In [82]:
print(reg.rsquared) 


AttributeError: 'RegularizedResults' object has no attribute 'rsquared'

In [80]:
modelo = 'renda ~ sexo + posse_de_veiculo + posse_de_imovel + estado_civil + educacao + tipo_renda + tipo_residencia + tempo_emprego + idade + qtd_filhos'
md = smf.ols(modelo, data = df)
reg = md.fit_regularized(method = 'elastic_net' 
                         , L1_wt = 0.0
                         , alpha = 0.001
                         , refit = True)
print(reg.params) 


[-2492.37755053  6011.30398915    55.39459201   361.5417912
  -109.33676004  -449.70943036  -479.51536042  -221.95227326
   440.73441986   170.27059771   776.87786601  -221.0888736
  -566.57494999   815.65017112 -1126.04140335   135.80397981
  -378.66132071  -468.84663763  -482.80759336    14.43005055
    -6.17963206   554.60046204    40.67595317   136.78991251]


In [83]:
print(reg.rsquared) 


AttributeError: 'RegularizedResults' object has no attribute 'rsquared'

In [ ]:
modelo = 'renda ~ sexo + posse_de_veiculo + posse_de_imovel + estado_civil + educacao + tipo_renda + tipo_residencia + tempo_emprego + idade + qtd_filhos'
md = smf.ols(modelo, data = df)
reg = md.fit_regularized(method = 'elastic_net' 
                         , L1_wt = 0.0
                         , alpha = 0.005
                         , refit = True)
print(reg.params) 

In [31]:
print(reg.rsquared)

AttributeError: 'RegularizedResults' object has no attribute 'rsquared'

In [32]:
modelo = 'renda ~ sexo + posse_de_veiculo + posse_de_imovel + estado_civil + educacao + tipo_renda + tipo_residencia + tempo_emprego + idade + qtd_filhos'
md = smf.ols(modelo, data = df)
reg = md.fit_regularized(method = 'elastic_net' 
                         , L1_wt = 0.0
                         , alpha = 0.01
                         , refit = True)
print(reg.params) 

[-1465.44800354  5719.69678217   114.11095004   321.4735527
  -112.57291263  -456.39615099  -464.85919084  -203.32908165
    24.1736204   -317.84164777   252.19055457  -626.27288382
   -91.11077608   743.657465    -182.9752404     90.43670285
  -449.93461349  -505.96522779  -243.02314707    19.09159925
   -76.76599515   553.28062976    33.29311302   111.23304551]


In [84]:
print(reg.rsquared) 

AttributeError: 'RegularizedResults' object has no attribute 'rsquared'

In [33]:
modelo = 'renda ~ sexo + posse_de_veiculo + posse_de_imovel + estado_civil + educacao + tipo_renda + tipo_residencia + tempo_emprego + idade + qtd_filhos'
md = smf.ols(modelo, data = df)
reg = md.fit_regularized(method = 'elastic_net' 
                         , L1_wt = 0.0
                         , alpha = 0.05
                         , refit = True)
print(reg.params) 

[-6.43443973e+02  4.78018140e+03  3.19292193e+02  2.28666321e+02
 -1.04912138e+02 -3.81508316e+02 -3.57053570e+02 -1.56322016e+02
  1.65332531e+00 -2.89112523e+02  1.87453984e+02 -3.88469470e+02
 -2.04643919e+01  5.50451859e+02 -3.99346286e+01 -9.82877401e-01
 -3.35566261e+02 -2.91139539e+02 -6.33553707e+01  4.91800602e+01
 -1.82142363e+01  5.46936148e+02  2.10026511e+01  8.42723816e+01]


In [85]:
print(reg.rsquared) 

AttributeError: 'RegularizedResults' object has no attribute 'rsquared'

In [34]:
modelo = 'renda ~ sexo + posse_de_veiculo + posse_de_imovel + estado_civil + educacao + tipo_renda + tipo_residencia + tempo_emprego + idade + qtd_filhos'
md = smf.ols(modelo, data = df)
reg = md.fit_regularized(method = 'elastic_net' 
                         , L1_wt = 0.0
                         , alpha = 0.1
                         , refit = True)
print(reg.params) 

[-3.30151955e+02  4.00779706e+03  4.51853431e+02  1.74421485e+02
 -9.40600587e+01 -2.95598920e+02 -2.67490825e+02 -1.32833123e+02
 -6.49473591e-02 -2.15148849e+02  1.89321319e+02 -2.36722603e+02
 -1.11843931e+01  4.28912506e+02 -2.03295118e+01 -3.16307040e+01
 -2.27958069e+02 -1.59995887e+02 -2.99765734e+01  4.53160522e+01
  2.70077531e+00  5.40600704e+02  1.75014311e+01  9.41880630e+01]


In [86]:
print(reg.rsquared) 

AttributeError: 'RegularizedResults' object has no attribute 'rsquared'

3. Faça o mesmo que no passo 2, com uma regressão *LASSO*. Qual método chega a um melhor resultado?


In [35]:
modelo = 'renda ~ sexo + posse_de_veiculo + posse_de_imovel + estado_civil + educacao + tipo_renda + tipo_residencia + tempo_emprego + idade + qtd_filhos'
md = smf.ols(modelo, data = df)
reg = md.fit_regularized(method = 'elastic_net' 
                         , L1_wt = 1
                         , alpha = 0.0
                         , refit = True)

reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  renda   R-squared:                       0.256
Model:                            OLS   Adj. R-squared:                  0.255
Method:                 Least Squares   F-statistic:                     178.0
Date:                Thu, 16 Feb 2023   Prob (F-statistic):               0.00
Time:                        19:51:48   Log-Likelihood:            -1.2879e+05
No. Observations:               12427   AIC:                         2.576e+05
Df Residuals:                   12403   BIC:                         2.578e+05
Df Model:                          24                                         
Covariance Type:            nonrobust                                         
====================================================================================================
                                       coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
Intercept                        -3089.4648   1004.807     -3.075      0.002   -5059.042   -1119.888
sexo[T.M]                         6051.4091    155.855     38.827      0.000    5745.909    6356.909
posse_de_veiculo[T.True]            48.3249    150.374      0.321      0.748    -246.432     343.082
posse_de_imovel[T.True]            368.1310    148.718      2.475      0.013      76.621     659.641
estado_civil[T.Separado]          -107.7415    297.434     -0.362      0.717    -690.759     475.276
estado_civil[T.Solteiro]          -440.7172    217.827     -2.023      0.043    -867.692     -13.743
estado_civil[T.União]             -476.5108    267.717     -1.780      0.075   -1001.279      48.257
estado_civil[T.Viúvo]             -217.1468    443.426     -0.490      0.624   -1086.331     652.038
educacao[T.Pós graduação]         1089.5486   1690.411      0.645      0.519   -2223.920    4403.017
educacao[T.Secundário]             607.3967    765.693      0.793      0.428    -893.481    2108.275
educacao[T.Superior completo]     1224.1466    768.570      1.593      0.111    -282.370    2730.664
educacao[T.Superior incompleto]    220.5729    827.105      0.267      0.790   -1400.681    1841.827
tipo_renda[T.Bolsista]           -1358.4674   2564.856     -0.530      0.596   -6385.984    3669.049
tipo_renda[T.Empresário]           820.7881    158.813      5.168      0.000     509.490    1132.087
tipo_renda[T.Pensionista]        -2686.8190   2565.510     -1.047      0.295   -7715.617    2341.980
tipo_renda[T.Servidor público]     138.0266    236.328      0.584      0.559    -325.212     601.265
tipo_residencia[T.Casa]           -297.8579    575.827     -0.517      0.605   -1426.567     830.852
tipo_residencia[T.Com os pais]    -387.2379    641.849     -0.603      0.546   -1645.361     870.885
tipo_residencia[T.Comunitário]    -458.9934   1120.531     -0.410      0.682   -2655.409    1737.422
tipo_residencia[T.Estúdio]          79.5298   1056.194      0.075      0.940   -1990.775    2149.834
tipo_residencia[T.Governamental]    81.8554    701.671      0.117      0.907   -1293.528    1457.239
tempo_emprego                      554.6942     11.044     50.225      0.000     533.046     576.343
idade                               42.0681      8.517      4.939      0.000      25.373      58.763
qtd_filhos                         141.3435     92.809      1.523      0.128     -40.577     323.264
==============================================================================
Omnibus:                    17567.671   Durbin-Watson:                   2.034
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          9881058.330
Skew:                           8.124   Prob(JB):                         0.00
Kurtosis:                     140.183   

In [36]:
modelo = 'renda ~ sexo + posse_de_veiculo + posse_de_imovel + estado_civil + educacao + tipo_renda + tipo_residencia + tempo_emprego + idade + qtd_filhos'
md = smf.ols(modelo, data = df)
reg = md.fit_regularized(method = 'elastic_net' 
                         , L1_wt = 1
                         , alpha = 0.001
                         , refit = True)

reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  renda   R-squared:                       0.256
Model:                            OLS   Adj. R-squared:                  0.255
Method:                 Least Squares   F-statistic:                     185.7
Date:                Thu, 16 Feb 2023   Prob (F-statistic):               0.00
Time:                        19:51:49   Log-Likelihood:            -1.2879e+05
No. Observations:               12427   AIC:                         2.576e+05
Df Residuals:                   12404   BIC:                         2.578e+05
Df Model:                          23                                         
Covariance Type:            nonrobust                                         
====================================================================================================
                                       coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
Intercept                        -3096.5916   1004.579     -3.082      0.002   -5065.722   -1127.461
sexo[T.M]                         6054.5197    155.613     38.908      0.000    5749.495    6359.545
posse_de_veiculo[T.True]            52.7992    149.861      0.352      0.725    -240.951     346.550
posse_de_imovel[T.True]            370.0340    148.620      2.490      0.013      78.716     661.352
estado_civil[T.Separado]                  0          0        nan        nan           0           0
estado_civil[T.Solteiro]          -431.9213    216.462     -1.995      0.046    -856.220      -7.623
estado_civil[T.União]             -467.4868    266.547     -1.754      0.079    -989.959      54.986
estado_civil[T.Viúvo]             -205.4525    442.234     -0.465      0.642   -1072.300     661.395
educacao[T.Pós graduação]         1096.6779   1690.238      0.649      0.516   -2216.450    4409.806
educacao[T.Secundário]             609.8679    765.636      0.797      0.426    -890.898    2110.634
educacao[T.Superior completo]     1225.5914    768.533      1.595      0.111    -280.853    2732.035
educacao[T.Superior incompleto]    225.1146    826.981      0.272      0.785   -1395.897    1846.126
tipo_renda[T.Bolsista]           -1344.8554   2564.491     -0.524      0.600   -6371.656    3681.945
tipo_renda[T.Empresário]           820.2528    158.801      5.165      0.000     508.979    1131.527
tipo_renda[T.Pensionista]        -2685.7260   2565.419     -1.047      0.295   -7714.345    2342.893
tipo_renda[T.Servidor público]     140.1449    236.247      0.593      0.553    -322.936     603.225
tipo_residencia[T.Casa]           -298.3653    575.805     -0.518      0.604   -1427.032     830.301
tipo_residencia[T.Com os pais]    -390.8470    641.749     -0.609      0.543   -1648.774     867.081
tipo_residencia[T.Comunitário]    -454.8269   1120.433     -0.406      0.685   -2651.049    1741.396
tipo_residencia[T.Estúdio]          78.1751   1056.150      0.074      0.941   -1992.044    2148.394
tipo_residencia[T.Governamental]    83.9765    701.622      0.120      0.905   -1291.311    1459.264
tempo_emprego                      554.7035     11.044     50.228      0.000     533.056     576.351
idade                               41.8833      8.502      4.926      0.000      25.219      58.548
qtd_filhos                         141.6480     92.802      1.526      0.127     -40.258     323.555
==============================================================================
Omnibus:                    17568.740   Durbin-Watson:                   2.034
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          9884252.657
Skew:                           8.125   Prob(JB):                         0.00
Kurtosis:                     140.205   

In [37]:
modelo = 'renda ~ sexo + posse_de_veiculo + posse_de_imovel + estado_civil + educacao + tipo_renda + tipo_residencia + tempo_emprego + idade + qtd_filhos'
md = smf.ols(modelo, data = df)
reg = md.fit_regularized(method = 'elastic_net' 
                         , L1_wt = 1
                         , alpha = 0.005
                         , refit = True)

reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  renda   R-squared:                       0.256
Model:                            OLS   Adj. R-squared:                  0.255
Method:                 Least Squares   F-statistic:                     185.7
Date:                Thu, 16 Feb 2023   Prob (F-statistic):               0.00
Time:                        19:51:49   Log-Likelihood:            -1.2879e+05
No. Observations:               12427   AIC:                         2.576e+05
Df Residuals:                   12404   BIC:                         2.578e+05
Df Model:                          23                                         
Covariance Type:            nonrobust                                         
====================================================================================================
                                       coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
Intercept                        -3096.5916   1004.579     -3.082      0.002   -5065.722   -1127.461
sexo[T.M]                         6054.5197    155.613     38.908      0.000    5749.495    6359.545
posse_de_veiculo[T.True]            52.7992    149.861      0.352      0.725    -240.951     346.550
posse_de_imovel[T.True]            370.0340    148.620      2.490      0.013      78.716     661.352
estado_civil[T.Separado]                  0          0        nan        nan           0           0
estado_civil[T.Solteiro]          -431.9213    216.462     -1.995      0.046    -856.220      -7.623
estado_civil[T.União]             -467.4868    266.547     -1.754      0.079    -989.959      54.986
estado_civil[T.Viúvo]             -205.4525    442.234     -0.465      0.642   -1072.300     661.395
educacao[T.Pós graduação]         1096.6779   1690.238      0.649      0.516   -2216.450    4409.806
educacao[T.Secundário]             609.8679    765.636      0.797      0.426    -890.898    2110.634
educacao[T.Superior completo]     1225.5914    768.533      1.595      0.111    -280.853    2732.035
educacao[T.Superior incompleto]    225.1146    826.981      0.272      0.785   -1395.897    1846.126
tipo_renda[T.Bolsista]           -1344.8554   2564.491     -0.524      0.600   -6371.656    3681.945
tipo_renda[T.Empresário]           820.2528    158.801      5.165      0.000     508.979    1131.527
tipo_renda[T.Pensionista]        -2685.7260   2565.419     -1.047      0.295   -7714.345    2342.893
tipo_renda[T.Servidor público]     140.1449    236.247      0.593      0.553    -322.936     603.225
tipo_residencia[T.Casa]           -298.3653    575.805     -0.518      0.604   -1427.032     830.301
tipo_residencia[T.Com os pais]    -390.8470    641.749     -0.609      0.543   -1648.774     867.081
tipo_residencia[T.Comunitário]    -454.8269   1120.433     -0.406      0.685   -2651.049    1741.396
tipo_residencia[T.Estúdio]          78.1751   1056.150      0.074      0.941   -1992.044    2148.394
tipo_residencia[T.Governamental]    83.9765    701.622      0.120      0.905   -1291.311    1459.264
tempo_emprego                      554.7035     11.044     50.228      0.000     533.056     576.351
idade                               41.8833      8.502      4.926      0.000      25.219      58.548
qtd_filhos                         141.6480     92.802      1.526      0.127     -40.258     323.555
==============================================================================
Omnibus:                    17568.740   Durbin-Watson:                   2.034
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          9884252.657
Skew:                           8.125   Prob(JB):                         0.00
Kurtosis:                     140.205   

In [38]:
modelo = 'renda ~ sexo + posse_de_veiculo + posse_de_imovel + estado_civil + educacao + tipo_renda + tipo_residencia + tempo_emprego + idade + qtd_filhos'
md = smf.ols(modelo, data = df)
reg = md.fit_regularized(method = 'elastic_net' 
                         , L1_wt = 1
                         , alpha = 0.01
                         , refit = True)

reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  renda   R-squared:                       0.256
Model:                            OLS   Adj. R-squared:                  0.255
Method:                 Least Squares   F-statistic:                     185.7
Date:                Thu, 16 Feb 2023   Prob (F-statistic):               0.00
Time:                        19:51:50   Log-Likelihood:            -1.2879e+05
No. Observations:               12427   AIC:                         2.576e+05
Df Residuals:                   12404   BIC:                         2.578e+05
Df Model:                          23                                         
Covariance Type:            nonrobust                                         
====================================================================================================
                                       coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
Intercept                        -3096.5916   1004.579     -3.082      0.002   -5065.722   -1127.461
sexo[T.M]                         6054.5197    155.613     38.908      0.000    5749.495    6359.545
posse_de_veiculo[T.True]            52.7992    149.861      0.352      0.725    -240.951     346.550
posse_de_imovel[T.True]            370.0340    148.620      2.490      0.013      78.716     661.352
estado_civil[T.Separado]                  0          0        nan        nan           0           0
estado_civil[T.Solteiro]          -431.9213    216.462     -1.995      0.046    -856.220      -7.623
estado_civil[T.União]             -467.4868    266.547     -1.754      0.079    -989.959      54.986
estado_civil[T.Viúvo]             -205.4525    442.234     -0.465      0.642   -1072.300     661.395
educacao[T.Pós graduação]         1096.6779   1690.238      0.649      0.516   -2216.450    4409.806
educacao[T.Secundário]             609.8679    765.636      0.797      0.426    -890.898    2110.634
educacao[T.Superior completo]     1225.5914    768.533      1.595      0.111    -280.853    2732.035
educacao[T.Superior incompleto]    225.1146    826.981      0.272      0.785   -1395.897    1846.126
tipo_renda[T.Bolsista]           -1344.8554   2564.491     -0.524      0.600   -6371.656    3681.945
tipo_renda[T.Empresário]           820.2528    158.801      5.165      0.000     508.979    1131.527
tipo_renda[T.Pensionista]        -2685.7260   2565.419     -1.047      0.295   -7714.345    2342.893
tipo_renda[T.Servidor público]     140.1449    236.247      0.593      0.553    -322.936     603.225
tipo_residencia[T.Casa]           -298.3653    575.805     -0.518      0.604   -1427.032     830.301
tipo_residencia[T.Com os pais]    -390.8470    641.749     -0.609      0.543   -1648.774     867.081
tipo_residencia[T.Comunitário]    -454.8269   1120.433     -0.406      0.685   -2651.049    1741.396
tipo_residencia[T.Estúdio]          78.1751   1056.150      0.074      0.941   -1992.044    2148.394
tipo_residencia[T.Governamental]    83.9765    701.622      0.120      0.905   -1291.311    1459.264
tempo_emprego                      554.7035     11.044     50.228      0.000     533.056     576.351
idade                               41.8833      8.502      4.926      0.000      25.219      58.548
qtd_filhos                         141.6480     92.802      1.526      0.127     -40.258     323.555
==============================================================================
Omnibus:                    17568.740   Durbin-Watson:                   2.034
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          9884252.657
Skew:                           8.125   Prob(JB):                         0.00
Kurtosis:                     140.205   

In [39]:
modelo = 'renda ~ sexo + posse_de_veiculo + posse_de_imovel + estado_civil + educacao + tipo_renda + tipo_residencia + tempo_emprego + idade + qtd_filhos'
md = smf.ols(modelo, data = df)
reg = md.fit_regularized(method = 'elastic_net' 
                         , L1_wt = 1
                         , alpha = 0.05
                         , refit = True)

reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  renda   R-squared:                       0.256
Model:                            OLS   Adj. R-squared:                  0.255
Method:                 Least Squares   F-statistic:                     203.4
Date:                Thu, 16 Feb 2023   Prob (F-statistic):               0.00
Time:                        19:51:50   Log-Likelihood:            -1.2879e+05
No. Observations:               12427   AIC:                         2.576e+05
Df Residuals:                   12406   BIC:                         2.578e+05
Df Model:                          21                                         
Covariance Type:            nonrobust                                         
====================================================================================================
                                       coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
Intercept                        -3039.0654    887.068     -3.426      0.001   -4777.856   -1300.274
sexo[T.M]                         6053.7153    155.293     38.982      0.000    5749.317    6358.114
posse_de_veiculo[T.True]            52.9788    149.814      0.354      0.724    -240.681     346.638
posse_de_imovel[T.True]            369.1964    148.375      2.488      0.013      78.358     660.035
estado_civil[T.Separado]                  0          0        nan        nan           0           0
estado_civil[T.Solteiro]          -433.1379    216.214     -2.003      0.045    -856.951      -9.325
estado_civil[T.União]             -467.9872    266.444     -1.756      0.079    -990.260      54.285
estado_civil[T.Viúvo]             -206.0106    442.167     -0.466      0.641   -1072.726     660.705
educacao[T.Pós graduação]         1096.1923   1690.069      0.649      0.517   -2216.606    4408.990
educacao[T.Secundário]             609.0835    765.491      0.796      0.426    -891.397    2109.564
educacao[T.Superior completo]     1224.7334    768.325      1.594      0.111    -281.304    2730.770
educacao[T.Superior incompleto]    223.2159    826.715      0.270      0.787   -1397.273    1843.705
tipo_renda[T.Bolsista]           -1345.4793   2564.280     -0.525      0.600   -6371.866    3680.908
tipo_renda[T.Empresário]           820.4070    158.741      5.168      0.000     509.249    1131.564
tipo_renda[T.Pensionista]        -2684.6924   2565.199     -1.047      0.295   -7712.881    2343.496
tipo_renda[T.Servidor público]     140.2518    236.027      0.594      0.552    -322.398     602.902
tipo_residencia[T.Casa]           -356.6972    320.289     -1.114      0.265    -984.513     271.119
tipo_residencia[T.Com os pais]    -448.8391    430.790     -1.042      0.297   -1293.254     395.576
tipo_residencia[T.Comunitário]    -513.1787   1012.990     -0.507      0.612   -2498.797    1472.440
tipo_residencia[T.Estúdio]                0          0        nan        nan           0           0
tipo_residencia[T.Governamental]          0          0        nan        nan           0           0
tempo_emprego                      554.7334     11.039     50.252      0.000     533.095     576.372
idade                               41.9383      8.489      4.940      0.000      25.299      58.578
qtd_filhos                         141.8230     92.769      1.529      0.126     -40.019     323.665
==============================================================================
Omnibus:                    17569.460   Durbin-Watson:                   2.034
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          9887314.962
Skew:                           8.125   Prob(JB):                         0.00
Kurtosis:                     140.226   

In [40]:
modelo = 'renda ~ sexo + posse_de_veiculo + posse_de_imovel + estado_civil + educacao + tipo_renda + tipo_residencia + tempo_emprego + idade + qtd_filhos'
md = smf.ols(modelo, data = df)
reg = md.fit_regularized(method = 'elastic_net' 
                         , L1_wt = 1
                         , alpha = 0.1
                         , refit = True)

reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  renda   R-squared:                       0.256
Model:                            OLS   Adj. R-squared:                  0.255
Method:                 Least Squares   F-statistic:                     213.6
Date:                Thu, 16 Feb 2023   Prob (F-statistic):               0.00
Time:                        19:51:51   Log-Likelihood:            -1.2879e+05
No. Observations:               12427   AIC:                         2.576e+05
Df Residuals:                   12407   BIC:                         2.578e+05
Df Model:                          20                                         
Covariance Type:            nonrobust                                         
====================================================================================================
                                       coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
Intercept                        -3045.3327    886.938     -3.434      0.001   -4783.869   -1306.796
sexo[T.M]                         6060.4101    154.622     39.195      0.000    5757.327    6363.493
posse_de_veiculo[T.True]            54.3342    149.781      0.363      0.717    -239.261     347.929
posse_de_imovel[T.True]            368.1142    148.352      2.481      0.013      77.321     658.908
estado_civil[T.Separado]                  0          0        nan        nan           0           0
estado_civil[T.Solteiro]          -426.5768    215.748     -1.977      0.048    -849.476      -3.677
estado_civil[T.União]             -461.7519    266.100     -1.735      0.083    -983.349      59.845
estado_civil[T.Viúvo]                     0          0        nan        nan           0           0
educacao[T.Pós graduação]         1114.4846   1689.560      0.660      0.510   -2197.315    4426.284
educacao[T.Secundário]             622.2158    764.948      0.813      0.416    -877.200    2121.632
educacao[T.Superior completo]     1238.1089    767.765      1.613      0.107    -266.829    2743.047
educacao[T.Superior incompleto]    237.1665    826.146      0.287      0.774   -1382.208    1856.541
tipo_renda[T.Bolsista]           -1330.3987   2563.995     -0.519      0.604   -6356.227    3695.429
tipo_renda[T.Empresário]           822.1520    158.692      5.181      0.000     511.091    1133.213
tipo_renda[T.Pensionista]        -2686.8528   2565.114     -1.047      0.295   -7714.875    2341.169
tipo_renda[T.Servidor público]     137.8330    235.963      0.584      0.559    -324.690     600.356
tipo_residencia[T.Casa]           -356.7778    320.279     -1.114      0.265    -984.574     271.018
tipo_residencia[T.Com os pais]    -448.0937    430.773     -1.040      0.298   -1292.476     396.288
tipo_residencia[T.Comunitário]    -521.6048   1012.797     -0.515      0.607   -2506.844    1463.634
tipo_residencia[T.Estúdio]                0          0        nan        nan           0           0
tipo_residencia[T.Governamental]          0          0        nan        nan           0           0
tempo_emprego                      554.8947     11.033     50.293      0.000     533.268     576.522
idade                               41.4922      8.434      4.919      0.000      24.960      58.025
qtd_filhos                         143.5140     92.695      1.548      0.122     -38.183     325.211
==============================================================================
Omnibus:                    17570.527   Durbin-Watson:                   2.034
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          9889529.637
Skew:                           8.126   Prob(JB):                         0.00
Kurtosis:                     140.242   

In [78]:
data = pd.read_csv('previsao_de_renda.csv')
data

,Unnamed: 0,data_ref,id_cliente,sexo,posse_de_veiculo,posse_de_imovel,qtd_filhos,tipo_renda,educacao,estado_civil,tipo_residencia,idade,tempo_emprego,qt_pessoas_residencia,renda
0,0,2015-01-01,15056,F,False,True,0,Empresário,Secundário,Solteiro,Casa,26,6.602740,1.0,8060.34
1,1,2015-01-01,9968,M,True,True,0,Assalariado,Superior completo,Casado,Casa,28,7.183562,2.0,1852.15
2,2,2015-01-01,4312,F,True,True,0,Empresário,Superior completo,Casado,Casa,35,0.838356,2.0,2253.89
3,3,2015-01-01,10639,F,False,True,1,Servidor público,Superior completo,Casado,Casa,30,4.846575,3.0,6600.77
4,4,2015-01-01,7064,M,True,False,0,Assalariado,Secundário,Solteiro,Governamental,33,4.293151,1.0,6475.97
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,14995,2016-03-01,16006,F,False,True,0,Empresário,Secundário,Solteiro,Casa,48,13.887671,1.0,7990.58
14996,14996,2016-03-01,3722,F,False,True,0,Pensionista,Superior completo,Solteiro,Casa,57,NaN,1.0,10093.45
14997,14997,2016-03-01,6194,F,True,True,0,Assalariado,Superior completo,Casado,Casa,45,7.832877,2.0,604.82
14998,14998,2016-03-01,4922,M,True,False,0,Empresário,Superior completo,Casado,Casa,36,4.298630,2.0,3352.27


4. Rode um modelo *stepwise*. Avalie o $R^2$ na vase de testes. Qual o melhor resultado?

In [79]:
data = pd.read_csv('previsao_de_renda.csv')
X = pd.DataFrame(data.data, columns=data.feature_names)
y = data.target


def stepwise_selection(X, y, 
                       initial_list=[], 
                       threshold_in=0.05, 
                       threshold_out = 0.05, 
                       verbose=True):

    included = list(initial_list)
    while True:
        changed=False
        # forward step
        excluded = list(set(X.columns)-set(included))
        new_pval = pd.Series(index=excluded, dtype=np.dtype('float64'))
        for new_column in excluded:
            model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included+[new_column]]))).fit()
            new_pval[new_column] = model.pvalues[new_column]
        best_pval = new_pval.min()
        if best_pval < threshold_in:
            best_feature = new_pval.index[new_pval.argmin()]
            included.append(best_feature)
            changed=True
            if verbose:
                 print('Add  {:30} with p-value {:.6}'.format(best_feature, best_pval))

        # backward step
        print("#############")
        print(included)
        model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included]))).fit()
        # use all coefs except intercept
        pvalues = model.pvalues.iloc[1:]
        worst_pval = pvalues.max() # null if pvalues is empty
        if worst_pval > threshold_out:
            changed=True
            worst_feature = pvalues.argmax()
            included.remove(worst_feature)
            if verbose:
                print('Drop {:30} with p-value {:.6}'.format(worst_feature, worst_pval))
        if not changed:
            break
    return included

variaveis = stepwise_selection(X, y)

print('resulting features:')
print(variaveis)

SyntaxError: invalid syntax (2009446148.py, line 2)

In [ ]:
reg_stepwise = sm.OLS(y, sm.add_constant(pd.DataFrame(X[variaveis]))).fit()
reg_stepwise.summary()

5. Compare os parâmetros e avalie eventuais diferenças. Qual modelo você acha o melhor de todos?
6. Partindo dos modelos que você ajustou, tente melhorar o $R^2$ na base de testes. Use a criatividade, veja se consegue inserir alguma transformação ou combinação de variáveis.
7. Ajuste uma árvore de regressão e veja se consegue um $R^2$ melhor com ela.